## Scraping and Cleaning NBA Game-By-Game Team Stats

The web provides numerous databases containing NBA player statistics. However, game-by-game team stats are harder to come by. Basketball-reference.com does provide this data, but in an encyclopedic fashion. That is, with a search and some clicking, team stats for any past matchup are available, but this info isn't available in a database. This project aimed to comb through the basketball-reference pages and create a database of team stats for the 2006-2007 NBA Season.

First: python packages related to managing and storing data are loaded.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pickle
import os
cwd = os.getcwd()

Specify a destination for the data.

In [2]:
os.chdir('C:\\Users\\JoCho\\Desktop\\Skool\\Project\\NBA')

Load packages related to web scraping.

In [3]:
from bs4 import BeautifulSoup
import requests
import itertools
import numpy.core.defchararray as np_f

from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from datetime import datetime

HTML/XML specific web scraping functions. Credit here to Colin O'Keefe and his very helpful Real Python article 'Practical Introduction to Web Scraping in Python'. (https://realpython.com/python-web-scraping-practical-introduction/)

In [4]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


Create a list of all the expected teams in the database. The same team initials are used that are used on the source site.

In [5]:
cities = ['BOS', 'CLE', 'HOU', 'GSW', 'BRK', 'IND', 'CHO', 'DET', 
          'PHI', 'WAS', 'MIA', 'ORL', 'MIL', 'NOP', 'MEM', 'ATL',
          'DAL', 'DEN', 'UTA', 'MIN', 'SAS', 'SAC', 'POR', 'PHO', 
          'SEA', 'TOR', 'CHI', 'NYK', 'OKC', 'LAC', 'LAL']

Iterate by date through the NBA season, grab each game's stats - and then split/dupicate those stats so there is a set for each team that played in the game. This requires hunting through the html source code to find the breaks you need the function to identify and then pull the content between those breaks. The results are dumped into a pandas data frame.

In [6]:
# iterating through pages by date and teams
# for purposes of this demo just grabbing first two months of the season

teams = []
team_wins = []
game_time = []
game_stats = []

start_date = dt.datetime(2006, 10,16)
end_date = dt.datetime(2006, 12,20)

total_days = (end_date - start_date).days + 1 #end date included

dash = '-'
for day_number in range(total_days):
    current_date = str((start_date + dt.timedelta(days = day_number)))
    current_date = current_date[0:10]
    current_date = current_date.replace("-","")
        
    #iterate through cities
    for town in range(0, len(cities)):
            page = requests.get('https://www.basketball-reference.com/boxscores/'+str(current_date)+'0'+
                                str(cities[town])+'.html')

            if is_good_response(page) == True:
                soup = BeautifulSoup(page.content, 'html.parser')
            else:
                continue
            
            #get matchup
            matchup = soup.title
            matchup = str(matchup)
            teams.append(matchup[7:-35])
            teams.append(matchup[7:-35])
            
            
            #get game time for sorting later
            time = soup.find_all(class_="scorebox_meta")
            time = str(time)
            time = time[34:42]
            game_time.append(time)
            game_time.append(time)

            
            
            #grab teams' current records
            temp = soup.select('h2')
    
            for i in range(0, len(temp)):
                temp[i] = str(temp[i])
                for j in range(0, len(temp[i])):
                    if temp[i][j] == "(":
                        team_wins.append(temp[i][j:j+5])
                
                        
            #grab game stats
            stats = []
            temp2 = soup.select('tfoot td')
            
            for i in range(0, len(temp2)):
                temp2[i] = str(temp2[i])
                temp2[i] = temp2[i][30:]
                
            stats = temp2
            for i in range(0, len(stats)):
                stats[i] = stats[i][0:-5]
                
            #append each games stats to master list
            game_stats.append([stats[1:19] + stats[21:35]])
            game_stats.append([stats[36:54] + stats[56:70]])
            
NBAstats = pd.DataFrame({'Matchup' : teams, 'Time' : game_time, 'Wins' : team_wins, 'Game Stats' : game_stats})
            
        
NBAstats.head()

,Matchup,Time,Wins,Game Stats
0,"Chicago Bulls at Miami Heat Box Score, October...","8:00 PM,",(1-0),"[[fg"">39, fga"">79, fg_pct"">.494, fg3"">7, fg3a""..."
1,"Chicago Bulls at Miami Heat Box Score, October...","8:00 PM,",(0-1),"[[fg"">25, fga"">65, fg_pct"">.385, fg3"">3, fg3a""..."
2,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(0-1),"[[fg"">40, fga"">77, fg_pct"">.519, fg3"">13, fg3a..."
3,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(1-0),"[[fg"">46, fga"">83, fg_pct"">.554, fg3"">6, fg3a""..."
4,New Orleans/Oklahoma City Hornets at Boston Ce...,"7:30 PM,",(1-0),"[[fg"">36, fga"">84, fg_pct"">.429, fg3"">1, fg3a""..."


Each entry in the 'Game Stats' column contains a list of items that contain unwanted characters (">). First, loop though the column to separate the lists into their separate entries.

In [7]:
for i in range(0, len(NBAstats)):
    NBAstats['Game Stats'][i] = NBAstats['Game Stats'][i][0]

In [8]:
NBAstats['Game Stats'][0]

['fg">39',
 'fga">79',
 'fg_pct">.494',
 'fg3">7',
 'fg3a">13',
 'fg3_pct">.538',
 'ft">23',
 'fta">32',
 'ft_pct">.719',
 'orb">13',
 'drb">36',
 'trb">49',
 'ast">22',
 'stl">10',
 'blk">5',
 'tov">16',
 'pf">21',
 'pts">108',
 'ts_pct">.580',
 'efg_pct">.538',
 'fg3a_per_fga_pct">.165',
 'fta_per_fga_pct">.405',
 'orb_pct">34.2',
 'drb_pct">90.0',
 'trb_pct">62.8',
 'ast_pct">56.4',
 'stl_pct">10.9',
 'blk_pct">10.4',
 'tov_pct">14.7',
 'usg_pct">100.0',
 'off_rtg">117.6',
 'def_rtg">71.9']

A function is crafted to split each entry to remove the unwanted HTML characters, and applied to each game stat list.

In [9]:
def splitStats(file):
    
    for line in range(0, len(file)):
        
        new = file[line].split('">')
        file[line] = new

In [10]:
for i in range(0, len(NBAstats['Game Stats'])):
    splitStats(NBAstats['Game Stats'][i])

After the function splitstats is applied, each game's stats are separated by category.

In [11]:
NBAstats['Game Stats'][0]

[['fg', '39'],
 ['fga', '79'],
 ['fg_pct', '.494'],
 ['fg3', '7'],
 ['fg3a', '13'],
 ['fg3_pct', '.538'],
 ['ft', '23'],
 ['fta', '32'],
 ['ft_pct', '.719'],
 ['orb', '13'],
 ['drb', '36'],
 ['trb', '49'],
 ['ast', '22'],
 ['stl', '10'],
 ['blk', '5'],
 ['tov', '16'],
 ['pf', '21'],
 ['pts', '108'],
 ['ts_pct', '.580'],
 ['efg_pct', '.538'],
 ['fg3a_per_fga_pct', '.165'],
 ['fta_per_fga_pct', '.405'],
 ['orb_pct', '34.2'],
 ['drb_pct', '90.0'],
 ['trb_pct', '62.8'],
 ['ast_pct', '56.4'],
 ['stl_pct', '10.9'],
 ['blk_pct', '10.4'],
 ['tov_pct', '14.7'],
 ['usg_pct', '100.0'],
 ['off_rtg', '117.6'],
 ['def_rtg', '71.9']]

Creating empty lists for each stat, and then filling these lists with the stats that were scraped and separated. These lists will then be appended to the original data frame NBAstats.

In [12]:
fg = []
fga = []
fg_pct = []
fg3 = []
fg3a = []
fg3_pct = []
ft = []
fta = []
ft_pct = []
orb = []
drb = []
trb = []
ast = []
stl = []
blk = []
tov = []
pf = []
points = []
ts_pct = []
efg_pct = []
fg3_per_pga_pct = []
fta_per_fga_pct = []
orb_pct = []
drb_pct = []
trb_pct = []
ast_pct = []
stl_pct = []
blk_pct = []
tov_pct = []
usg_pct = []
off_rtg = []
def_rtg = []

for i in range(0, len(NBAstats.index)):
    fg.append(NBAstats['Game Stats'][i][0][1])
    fga.append(NBAstats['Game Stats'][i][1][1])
    fg_pct.append(NBAstats['Game Stats'][i][2][1])
    fg3.append(NBAstats['Game Stats'][i][3][1])
    fg3a.append(NBAstats['Game Stats'][i][4][1])
    fg3_pct.append(NBAstats['Game Stats'][i][5][1])
    ft.append(NBAstats['Game Stats'][i][6][1])
    fta.append(NBAstats['Game Stats'][i][7][1])
    ft_pct.append(NBAstats['Game Stats'][i][8][1])
    orb.append(NBAstats['Game Stats'][i][9][1])
    drb.append(NBAstats['Game Stats'][i][10][1])
    trb.append(NBAstats['Game Stats'][i][11][1])
    ast.append(NBAstats['Game Stats'][i][12][1])
    stl.append(NBAstats['Game Stats'][i][13][1])
    blk.append(NBAstats['Game Stats'][i][14][1])
    tov.append(NBAstats['Game Stats'][i][15][1])
    pf.append(NBAstats['Game Stats'][i][16][1])
    points.append(NBAstats['Game Stats'][i][17][1])
    ts_pct.append(NBAstats['Game Stats'][i][18][1])
    efg_pct.append(NBAstats['Game Stats'][i][19][1])
    fg3_per_pga_pct.append(NBAstats['Game Stats'][i][20][1])
    fta_per_fga_pct.append(NBAstats['Game Stats'][i][21][1])
    orb_pct.append(NBAstats['Game Stats'][i][22][1])
    drb_pct.append(NBAstats['Game Stats'][i][23][1])
    trb_pct.append(NBAstats['Game Stats'][i][24][1])
    ast_pct.append(NBAstats['Game Stats'][i][25][1])
    stl_pct.append(NBAstats['Game Stats'][i][26][1])
    blk_pct.append(NBAstats['Game Stats'][i][27][1])
    tov_pct.append(NBAstats['Game Stats'][i][28][1])
    usg_pct.append(NBAstats['Game Stats'][i][29][1])
    off_rtg.append(NBAstats['Game Stats'][i][30][1])
    def_rtg.append(NBAstats['Game Stats'][i][31][1])
    


Add the lists to NBAstats and have a look.

In [13]:
NBAstats.insert(0, 'FGM', fg)
NBAstats.insert(0, 'FGA', fga)
NBAstats.insert(0, 'FG %.', fg_pct)
NBAstats.insert(0, '3PT', fg3)
NBAstats.insert(0, '3PT Att.', fg3a)
NBAstats.insert(0, '3PT %', fg3_pct)
NBAstats.insert(0, 'FT', ft)
NBAstats.insert(0, 'FTA', fta)
NBAstats.insert(0, 'FT %', ft_pct)
NBAstats.insert(0, 'Off Reb', orb)
NBAstats.insert(0, 'Def Reb', drb)
NBAstats.insert(0, 'Total Reb', trb)
NBAstats.insert(0, 'Asst', ast)
NBAstats.insert(0, 'Steals', stl)
NBAstats.insert(0, 'Blocks', blk)
NBAstats.insert(0, 'TO', tov)
NBAstats.insert(0, 'PF', pf)
NBAstats.insert(0, 'Points', points)
NBAstats.insert(0, 'True Shot %', ts_pct)
NBAstats.insert(0, 'Eff. FG %', efg_pct)
NBAstats.insert(0, '3PT Att. Rate', fg3_per_pga_pct)
NBAstats.insert(0, 'FT per FGA', fta_per_fga_pct)
NBAstats.insert(0, 'Off Reb %', orb_pct)
NBAstats.insert(0, 'Def Reb %', drb_pct)
NBAstats.insert(0, 'Total Reb %', trb_pct)
NBAstats.insert(0, 'Asst %', ast_pct)
NBAstats.insert(0, 'Steal %', stl_pct)
NBAstats.insert(0, 'Block %', blk_pct)
NBAstats.insert(0, 'TO %', tov_pct)
NBAstats.insert(0, 'USG', usg_pct)
NBAstats.insert(0, 'Off Rtg', off_rtg)
NBAstats.insert(0, 'Def Rtg', def_rtg)

NBAstats.head()

,Def Rtg,Off Rtg,USG,TO %,Block %,Steal %,Asst %,Total Reb %,Def Reb %,Off Reb %,...,3PT %,3PT Att.,3PT,FG %.,FGA,FGM,Matchup,Time,Wins,Game Stats
0,71.9,117.6,100.0,14.7,10.4,10.9,56.4,62.8,90.0,34.2,...,.538,13,7,.494,79,39,"Chicago Bulls at Miami Heat Box Score, October...","8:00 PM,",(1-0),"[[fg, 39], [fga, 79], [fg_pct, .494], [fg3, 7]..."
1,117.6,71.9,100.0,21.9,4.5,6.5,52.0,37.2,65.8,10.0,...,.176,17,3,.385,65,25,"Chicago Bulls at Miami Heat Box Score, October...","8:00 PM,",(0-1),"[[fg, 25], [fga, 65], [fg_pct, .385], [fg3, 3]..."
2,114.0,106.0,100.0,19.9,7.0,7.0,72.5,40.3,67.6,11.4,...,.433,30,13,.519,77,40,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(0-1),"[[fg, 40], [fga, 77], [fg_pct, .519], [fg3, 13..."
3,106.0,114.0,100.0,17.6,2.1,11.0,65.2,59.7,88.6,32.4,...,.500,12,6,.554,83,46,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(1-0),"[[fg, 46], [fga, 83], [fg_pct, .554], [fg3, 6]..."
4,86.4,90.3,100.0,13.2,6.8,6.0,47.2,46.9,78.3,19.2,...,.091,11,1,.429,84,36,New Orleans/Oklahoma City Hornets at Boston Ce...,"7:30 PM,",(1-0),"[[fg, 36], [fga, 84], [fg_pct, .429], [fg3, 1]..."


Looking good so far. The 'USG' stat is 'Usage Percentage'. Basketball reference defines this as the percentage of team plays that involved a given player. Because were are looking at team stats, USG necessarily always totals to 100%. In other words, USG is meaningless in the team context, so its dropped and the column names are adjusted accordingly.

In [14]:
NBAstats = NBAstats.drop(['USG'], axis=1)

In [15]:
col_names = ['Def Rtg', 'Off Rtg', 'TO %', 'Block %', 'Steal %', 'Asst %', 'Total Reb %', 'Def Reb %', 'Off Reb %',
             'FT per FGA', '3PT Att. Rate', 'Eff. FG %', 'True Shot %', 'PF', 'TO', 'Blocks', 'Steals', 'Asst',
             'Total Reb', 'Def Reb', 'Off Reb', 'FT %', 'FTA', 'FT', '3PT %', '3PT Att.', '3PT', 'FG %.', 'FGA', 'FGM']

In [16]:
for col in col_names:
    NBAstats[col] = pd.to_numeric(NBAstats[col], errors='coerce')
NBAstats.head()

,Def Rtg,Off Rtg,TO %,Block %,Steal %,Asst %,Total Reb %,Def Reb %,Off Reb %,FT per FGA,...,3PT %,3PT Att.,3PT,FG %.,FGA,FGM,Matchup,Time,Wins,Game Stats
0,71.9,117.6,14.7,10.4,10.9,56.4,62.8,90.0,34.2,0.405,...,0.538,13,7,0.494,79,39,"Chicago Bulls at Miami Heat Box Score, October...","8:00 PM,",(1-0),"[[fg, 39], [fga, 79], [fg_pct, .494], [fg3, 7]..."
1,117.6,71.9,21.9,4.5,6.5,52.0,37.2,65.8,10.0,0.338,...,0.176,17,3,0.385,65,25,"Chicago Bulls at Miami Heat Box Score, October...","8:00 PM,",(0-1),"[[fg, 25], [fga, 65], [fg_pct, .385], [fg3, 3]..."
2,114.0,106.0,19.9,7.0,7.0,72.5,40.3,67.6,11.4,0.221,...,0.433,30,13,0.519,77,40,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(0-1),"[[fg, 40], [fga, 77], [fg_pct, .519], [fg3, 13..."
3,106.0,114.0,17.6,2.1,11.0,65.2,59.7,88.6,32.4,0.289,...,0.500,12,6,0.554,83,46,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(1-0),"[[fg, 46], [fga, 83], [fg_pct, .554], [fg3, 6]..."
4,86.4,90.3,13.2,6.8,6.0,47.2,46.9,78.3,19.2,0.393,...,0.091,11,1,0.429,84,36,New Orleans/Oklahoma City Hornets at Boston Ce...,"7:30 PM,",(1-0),"[[fg, 36], [fga, 84], [fg_pct, .429], [fg3, 1]..."


The 'Matchup' column contains strings holding three important pieces of information - home team, away team and game date. 

In [17]:
NBAstats['Matchup'][0]

'Chicago Bulls at Miami Heat Box Score, October 31, 2006'

To parse this info, first split the game time off the end of the string, convert it to a datetime object, and add it to a new column ('Time').

In [18]:
game_start = []
for i in range(0, len(NBAstats['Time'])):
    if NBAstats['Time'][i][-1] == 'P':
        NBAstats['Time'][i] = NBAstats['Time'][i]+'M'
    if NBAstats['Time'][i][-1] == ',':
        NBAstats['Time'][i] = NBAstats['Time'][i][0:-1]
    game_start.append(datetime.strptime(NBAstats['Time'][i], '%H:%M %p'))

C:\Users\JoCho\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
NBAstats['Time'][0]

'8:00 PM'

Then load the game dates into a column of their own.

In [20]:
dates = []
dates.append(NBAstats['Matchup'].str.split(',',1))
dates = dates[0]
for i in range(0, len(dates)):
    dates[i] = dates[i][1]
dates = pd.to_datetime(dates)
dates[0]

Timestamp('2006-10-31 00:00:00')

Finally, the home and away teams will be placed into their own column. The trick here is that the away team is always listed first, and that's the way the stats fall in the data frame. Row 0 is the away team stats from the first game of the season, row 1 is the home team's stats from that game, row 2 is the away team's stats from the season's second game, and so on...

In [21]:
towns = ['Boston', 'Cleveland', 'Houston', 'Golden State', 'Brooklyn', 'Indiana', 'Charlotte', 'Detroit', 
          'Philadelphia', 'Washington', 'Miami', 'Orlando', 'Milwaukee', 'New Orleans', 'Memphis', 'Atlanta',
          'Dallas', 'Denver', 'Utah', 'Minnesota', 'San Antonio', 'Sacramento', 'Portland', 'Phoenix', 'Seattle',
          'Toronto', 'Chicago', 'New York', 'Oklahoma City', 'Los Angeles Clippers', 'Los Angeles Lakers']

In [22]:
away_home = []
away_home.append(NBAstats['Matchup'].str.split(' '))

away_home = away_home[0]
for i in range(0, len(away_home)):
    if i%2 == 0:
        if (away_home[i][0] == 'Los' or away_home[i][0] == 'New' or away_home[i][0] == 'San' or away_home[i][0] == 'Oklahoma' or 
            away_home[i][0] == 'Golden'):
            away_home[i] = away_home[i][2]
        
        else:
            away_home[i] = away_home[i][1]

for i in range(0, len(away_home)):
    if i%2 != 0:
        if (away_home[i][-6] == 'Lakers' or away_home[i][-6] == 'Clippers' or away_home[i][-6] == 'Pelicans' or 
            away_home[i][-6] == 'Spurs' or away_home[i][-6] == 'Thunder' or away_home[i][-6] == 'Knicks' or 
            away_home[i][-6] == 'Warriors'):
            away_home[i] = away_home[i][-6]
        else:
            away_home[i] = away_home[i][-6]
away_home[0:3]

0    Bulls
1     Heat
2     Suns
Name: Matchup, dtype: object

In [23]:
NBAstats['Team'] = away_home

Let's admire this thing of beauty!

In [24]:
NBAstats.head()

,Def Rtg,Off Rtg,TO %,Block %,Steal %,Asst %,Total Reb %,Def Reb %,Off Reb %,FT per FGA,...,3PT Att.,3PT,FG %.,FGA,FGM,Matchup,Time,Wins,Game Stats,Team
0,71.9,117.6,14.7,10.4,10.9,56.4,62.8,90.0,34.2,0.405,...,13,7,0.494,79,39,"Chicago Bulls at Miami Heat Box Score, October...",8:00 PM,(1-0),"[[fg, 39], [fga, 79], [fg_pct, .494], [fg3, 7]...",Bulls
1,117.6,71.9,21.9,4.5,6.5,52.0,37.2,65.8,10.0,0.338,...,17,3,0.385,65,25,"Chicago Bulls at Miami Heat Box Score, October...",8:00 PM,(0-1),"[[fg, 25], [fga, 65], [fg_pct, .385], [fg3, 3]...",Heat
2,114.0,106.0,19.9,7.0,7.0,72.5,40.3,67.6,11.4,0.221,...,30,13,0.519,77,40,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(0-1),"[[fg, 40], [fga, 77], [fg_pct, .519], [fg3, 13...",Suns
3,106.0,114.0,17.6,2.1,11.0,65.2,59.7,88.6,32.4,0.289,...,12,6,0.554,83,46,"Phoenix Suns at Los Angeles Lakers Box Score, ...",10:30 PM,(1-0),"[[fg, 46], [fga, 83], [fg_pct, .554], [fg3, 6]...",Lakers
4,86.4,90.3,13.2,6.8,6.0,47.2,46.9,78.3,19.2,0.393,...,11,1,0.429,84,36,New Orleans/Oklahoma City Hornets at Boston Ce...,7:30 PM,(1-0),"[[fg, 36], [fga, 84], [fg_pct, .429], [fg3, 1]...",City


Save the data to a pickle file, go outside and shoot some celebratory hoops!

In [589]:
NBAstats.to_pickle('NBAstats2006_07.pickle')